In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# read in all the words
words = open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [16]:
# build the dataset
# block size is the context length: how many chars do we take to predict the next one
block_size = 3 
x, y =[],[]
for w in words[:5]:
    print(w)
    context = [0] * block_size # initializing the block with ... padding
    for ch in w + '.': #to mark the ending of a word
        ix = stoi[ch]
        x.append(context) # this is where it is intially padded with dots....
        y.append(ix) # index of the next character
        print(''.join(itos[i] for i in context),'---->',itos[ix])
        context = context[1:] + [ix] #crop and append the character for next iteration(decrease the padded dots)
    
x = torch.tensor(x)
y = torch.tensor(y)
print(x[1,:])
print(y[1])
#from the above print example you can see that X is a possible context
#before the next character which is y

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .
tensor([0, 0, 5])
tensor(13)


In [25]:
x.shape, x.dtype, y.shape, y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [17]:
c = torch.rand((27,2))

In [19]:
c[5]

tensor([0.1758, 0.5261])

In [26]:
c[[5,6,7]] # torch has a functionality which lets u index with a list
c[x]

tensor([[[0.7556, 0.9409],
         [0.7556, 0.9409],
         [0.7556, 0.9409]],

        [[0.7556, 0.9409],
         [0.7556, 0.9409],
         [0.1758, 0.5261]],

        [[0.7556, 0.9409],
         [0.1758, 0.5261],
         [0.6099, 0.8225]],

        [[0.1758, 0.5261],
         [0.6099, 0.8225],
         [0.6099, 0.8225]],

        [[0.6099, 0.8225],
         [0.6099, 0.8225],
         [0.0095, 0.9829]],

        [[0.7556, 0.9409],
         [0.7556, 0.9409],
         [0.7556, 0.9409]],

        [[0.7556, 0.9409],
         [0.7556, 0.9409],
         [0.4164, 0.7279]],

        [[0.7556, 0.9409],
         [0.4164, 0.7279],
         [0.7187, 0.0331]],

        [[0.4164, 0.7279],
         [0.7187, 0.0331],
         [0.3101, 0.6418]],

        [[0.7187, 0.0331],
         [0.3101, 0.6418],
         [0.1881, 0.2812]],

        [[0.3101, 0.6418],
         [0.1881, 0.2812],
         [0.3101, 0.6418]],

        [[0.1881, 0.2812],
         [0.3101, 0.6418],
         [0.0095, 0.9829]],

    

In [27]:
c[x].shape

torch.Size([32, 3, 2])

In [28]:
# embedding(first layer)
emb = c[x]

In [29]:
# hidden layer
w1 = torch.randn((6,100))
b1 = torch.randn(100)


In [34]:
# view() is used to reshape (32,3,2) in to (32,6) for matrix multiplication purpose
h =torch.tanh(emb.view(emb.shape[0],6) @ w1 + b1)
h.shape

torch.Size([32, 100])

In [35]:
# final layer
w2 = torch.randn((100,27))
b2 = torch.randn(27)

In [37]:
logits = h@w2 + b2
logits.shape

torch.Size([32, 27])

In [40]:
# softmax of final layer
counts = logits.exp()
prob = counts / counts.sum(1, keepdims = True) 
prob[0].sum() # therefore nomrlaized.

tensor(1.)